# Functional MRI Preprocessing Steps

In [16]:
# import required packages
import os
import numpy as np
import pandas as pd

In [17]:
# function for viewing full dataframes

def print_full_df(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')


In [18]:
def append_column(df, dir, column_name, search_phrase='.nii'):
    '''
    This function appends column of filepaths to a dataframe matching by subject_id. 

    Files within folders must contain the subject id string pattern. 

    INPUTS:
        1. main_df          - the dataframe to append columns to
        2. folder_name      - folder containing files/paths to append
        3. column_name      - name of the column to append
        4. search_phrase    - text pattern to look for inside specified folder (default '.nii')

    '''
    # search directory for phrase and store as a list
    filelist = !find "{fmri_dir}/{dir}" ~+ -type f -name "*{search_phrase}*"

    # convert list to a dataframe and create a column matching subject_id
    tmp_df = pd.DataFrame({'subject_id':"", column_name:filelist})
    tmp_df['subject_id'] = tmp_df[column_name].str.extract(r'(\w{3}_\d{3}_m\d{2})')

    # append the new filepaths by matching subject_id 
    df = pd.merge(df, tmp_df, on='subject_id')

    return df

In [19]:
def append_anat_column(df, dir, column_name, search_phrase='.nii'):
    '''
    This function appends column of filepaths to a dataframe matching by subject_id. 

    Files within folders must contain the subject id string pattern. 

    INPUTS:
        1. main_df          - the dataframe to append columns to
        2. folder_name      - folder containing files/paths to append
        3. column_name      - name of the column to append
        4. search_phrase    - text pattern to look for inside specified folder (default '.nii')

    '''
    # search directory for phrase and store as a list
    filelist = !find "{anat_dir}/{dir}" ~+ -type f -name "*{search_phrase}*"

    # convert list to a dataframe and create a column matching subject_id
    tmp_df = pd.DataFrame({'subject_id':"", column_name:filelist})
    tmp_df['subject_id'] = tmp_df[column_name].str.extract(r'(\w{3}_\d{3}_m\d{2})')

    # append the new filepaths by matching subject_id 
    df = pd.merge(df, tmp_df, on='subject_id')

    return df

In [20]:
# define parent directories for func and struct data
anat_dir = '/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden'
fmri_dir = '/media/forest/wd_1/data_CLMS/functional/7T_func'

In [21]:
# create the initial dataframe with subject_id and raw data filepaths
raw_filelist = !find "{fmri_dir}/0_raw" ~+ -type f -name "*.nii*"
df_pp_fmri = pd.DataFrame({'subject_id':"", 'fp_raw':raw_filelist})
df_pp_fmri['subject_id'] = df_pp_fmri['fp_raw'].str.extract(r'(\w{3}_\d{3}_m\d{2})')

In [22]:
## Create code to extract volume #240 from each subject's raw fmri data
for subject, file in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_raw']):
    print(f'fslroi {file} {fmri_dir}/3_mid_volume/{subject}_func_middle_volume 240 1 &&')

fslroi /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aab_026_m00_7T_FMRI_raw.nii /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aab_026_m00_func_middle_volume 240 1 &&
fslroi /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aac_027_m00_7T_FMRI_raw.nii /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aac_027_m00_func_middle_volume 240 1 &&
fslroi /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aac_027_m24_7T_FMRI_raw.nii /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aac_027_m24_func_middle_volume 240 1 &&
fslroi /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aad_028_m00_7T_FMRI_raw.nii /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aad_028_m00_func_middle_volume 240 1 &&
fslroi /media/forest/wd_1/data_CLMS/functional/7T_func/0_raw/aad_028_m24_7T_FMRI_raw.nii /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aad_028_m24_func_middle_volume 240 1 &&
fslroi /media/forest/wd_1/data_CLMS/functional/7T_

In [23]:
## Append to df, check shape which should be 64x3
df_pp_fmri = append_column(df_pp_fmri, '3_mid_volume', 'fp_middle_volume', '.nii')

In [24]:
## Create code to skullstrip the middle volume data--use conservative settings in BET (same as FEAT pipeline) and visually inspect results
for subject, file in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_middle_volume']):
    print(f'bet2 {file} {fmri_dir}/5_mid_volume_SS/{subject}_func_middle_volume_skullstripped -f 0.3 &&')

bet2 /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aab_026_m00_func_middle_volume.nii.gz /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aab_026_m00_func_middle_volume_skullstripped -f 0.3 &&
bet2 /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aac_027_m00_func_middle_volume.nii.gz /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aac_027_m00_func_middle_volume_skullstripped -f 0.3 &&
bet2 /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aac_027_m24_func_middle_volume.nii.gz /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aac_027_m24_func_middle_volume_skullstripped -f 0.3 &&
bet2 /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aad_028_m00_func_middle_volume.nii.gz /media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aad_028_m00_func_middle_volume_skullstripped -f 0.3 &&
bet2 /media/forest/wd_1/data_CLMS/functional/7T_func/3_mid_volume/aad_028_m24_func_middle_volume.nii.gz /med

In [25]:
## append to df, check shape which should be 64x4
df_pp_fmri = append_column(df_pp_fmri, '5_mid_volume_SS', 'fp_middle_volume_SS', '.nii')

In [26]:
df_pp_fmri

,subject_id,fp_raw,fp_middle_volume,fp_middle_volume_SS
0,aab_026_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
1,aac_027_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
2,aac_027_m24,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
3,aad_028_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
4,aad_028_m24,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
...,...,...,...,...
59,xyz_024_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
60,xyz_024_m24,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
61,aal_036_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...
62,bbi_059_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...


# Boundary Based Registration (fmri to anatomical transformation)

Required inputs: 
- functional image--use single-volume (volume # 240) from functional scan
    - middle volume in `func/3_mid_volume`
    - skullstripped in `func/5_mid_volume_SS`
- t1 wholehead image--use `anat/1_robustfov`
- t1 brain only--use `anat/2_skullstripped` 
- output path--use `func/7_fmri2anat_xfm`

In [27]:
# get filepaths for the T1 images and append these to the column
df_pp_fmri = append_anat_column(df_pp_fmri, '1_robustfov_fslreorient2std', 'fp_t1_wholebrain', '.nii')

In [28]:
df_pp_fmri

,subject_id,fp_raw,fp_middle_volume,fp_middle_volume_SS,fp_t1_wholebrain
0,aab_026_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
1,aac_027_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
2,aac_027_m24,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
3,aad_028_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
4,aad_028_m24,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
...,...,...,...,...,...
59,xyz_024_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
60,xyz_024_m24,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
61,aal_036_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
62,bbi_059_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...


In [29]:
df_pp_fmri = append_anat_column(df_pp_fmri, '2_skullstripped', 'fp_t1_skullstripped', 'BrainExtractionBrain.nii')

In [30]:
df_pp_fmri

,subject_id,fp_raw,fp_middle_volume,fp_middle_volume_SS,fp_t1_wholebrain,fp_t1_skullstripped
0,aab_026_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
1,aac_027_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
2,aac_027_m24,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
3,aad_028_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
4,aad_028_m24,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
...,...,...,...,...,...,...
59,xyz_024_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
60,xyz_024_m24,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
61,aal_036_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
62,bbi_059_m00,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/functional/7T_fun...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...


In [32]:
for subject, fmri_file, t1_wholebrain, t1_skullstripped in zip(df_pp_fmri['subject_id'], df_pp_fmri['fp_middle_volume_SS'], df_pp_fmri['fp_t1_wholebrain'], df_pp_fmri['fp_t1_skullstripped']):
    print(f'epi_reg --epi={fmri_file} --t1={t1_wholebrain} --t1brain={t1_skullstripped} --out={fmri_dir}/7_fmri2anat_xfm/{subject}_fmri2anat &')

epi_reg --epi=/media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aab_026_m00_func_middle_volume_skullstripped.nii.gz --t1=/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/1_robustfov_fslreorient2std/aab_026_m00_7T_mprage_uniden_robustfov.nii.gz --t1brain=/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped/aab_026_m00_T1_mprage_uniden_skullstripped_after_robustfov_BrainExtractionBrain.nii.gz --out=/media/forest/wd_1/data_CLMS/functional/7T_func/7_fmri2anat_xfm/aab_026_m00_fmri2anat &
epi_reg --epi=/media/forest/wd_1/data_CLMS/functional/7T_func/5_mid_volume_SS/aac_027_m00_func_middle_volume_skullstripped.nii.gz --t1=/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/1_robustfov_fslreorient2std/aac_027_m00_7T_mprage_uniden_robustfov.nii.gz --t1brain=/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped/aac_027_m00_T1_mprage_uniden_skullstripped_after_robustfov_BrainExtractionBrain.nii.gz --out=/media/forest/wd_1/data_CLMS

- Can run 15 in parallel with ~30 GB memory usage 
- spot check by overlaying resulting transformed nifti image onto the anatomical image
- generated code for fslview in below cell: 